In [2]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [3]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [4]:
def getInfo(filesGrp, img_path):

  dslabel = {
    "id":"",
    "originalFileJPG":"",
    "originalFileGrp":[],
    "destFolder":"",
    "GeoTagInfo":{
      "coords":[],
      "datetime_original":"",
      "gps_altitude":0,
      "gps_altitude_ref":""    
    },
    "dest":{
      "rgb":"",
      "ndvi":"",
      "thumb":"",
    },
    "info":{
      "course":"",
      "area":"",
      "desc":"",
      "alt":""
    },
    "label":[],
    "annotation":[]
  }

# Remove Drive letter with root path
  dslabel['originalFileJPG'] = img_path[3:]
  dslabel['originalFileGrp'] = [x[3:] for x in filesGrp  if ('.JPG' not in x)]

  with open(img_path, 'rb') as src:
    img = exifimg(src)

  coords = [
    decimal_coords(img.gps_longitude,
    img.gps_longitude_ref),
      decimal_coords(img.gps_latitude,
    img.gps_latitude_ref)
  ]

  dslabel['id'] = img.datetime_original.replace(" ","").replace(":","") + "_{:.10f}".format(coords[0]).replace(".","") + "_{:.10f}".format(coords[1]).replace(".","") + "_{:f}".format(img.gps_altitude).replace(".","")
  dslabel['GeoTagInfo']['coords'] = coords
  dslabel['GeoTagInfo']['datetime_original'] = img.datetime_original
  dslabel['GeoTagInfo']['gps_altitude'] = img.gps_altitude
  dslabel['GeoTagInfo']['gps_altitude_ref'] = img.gps_altitude_ref
  
  return dslabel

In [5]:
def getNDVIimg(filesGrp):


  with rasterio.open([ x for x in filesGrp if ('_MS_R' in x) ][0]) as src:
      band_red = src.read(1).astype(float)/65536.

  with rasterio.open([ x for x in filesGrp if ('_MS_NIR' in x) ][0]) as src:
      band_nir = src.read(1).astype(float)/65536.

  np.seterr(divide='ignore', invalid='ignore')

  # Calculate NDVI
  ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float))

  ndvi -= ndvi.min() # ensure the minimal value is 0.0
  ndvi /= ndvi.max() # maximum value in image is now 1.0

  cm = plt.cm.get_cmap('RdYlGn')
  ndvi_cm = cm(ndvi)

  ndvi_img = cv2.normalize(ndvi_cm[:,:,:3], None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8)

  return ndvi_img

In [6]:
def getalignedRGB(filesGrp_, ndvi_img):
  RGBimg = np.asarray(Image.open(filesGrp_[0]))

  # RGBimg = np.asarray(Image.open(filesGrp_[0]).resize((ndvi_img.shape[1],ndvi_img.shape[0])))
  # features0 = FeatureExtraction(RGBimg)
  # features1 = FeatureExtraction(ndvi_img)

  # matches = feature_matching(features0, features1)

  # print('{} number of matched has been found! in {}'.format(len(matches), filesGrp_[0]))

  # if len(matches) < 10:
  #    raise Exception("Not Enough Matches")

  # H, _ = cv2.findHomography( features0.matched_pts, features1.matched_pts, cv2.RANSAC, 5.0)

  H = np.array([[ 5.93209530e-01, -8.35074813e-03, -2.21474434e+02],
       [ 1.32198817e-02,  5.96460559e-01, -2.01042419e+02],
       [ 1.58520522e-06,  1.69424781e-06,  1.00000000e+00]])

  h, w, c = ndvi_img.shape
  RGBimg_tuned = cv2.warpPerspective(RGBimg, H, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

  return RGBimg_tuned 

In [7]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS'

course_list = ['청도','포항','중문']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
}

folder_type = ['rgb','ndvi','thumb']

In [8]:
course_ids.get('청도')

'MGC002'

In [9]:
img_folders_candidate =glob.glob(os.path.join(img_folder,'*'))
img_folders_candidate

['D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본',
 'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완',
 'D:\\original\\2023_0816_포항CC_오전맑음오후구름가득_데이터 1차 분류본_완']

In [10]:
img_folder = img_folders_candidate[0]

In [11]:
files = glob.glob(os.path.join(img_folder,'**/DJI_*.JPG'), recursive=True)

len(files)

8320

In [12]:


for idx, file_ in enumerate(files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  course_id = course_ids.get(course)
  target_date = os.path.split(file_)[-1].split('_')[1][:8]

  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3]

  if len(info[5].split('_')) ==3 :
    desc = info[5].split('_')[-1]
  else:
    desc = "미기재"
    
  alt =  info[4].split('_')[-1]

  # print(out_folder,course,target_date,area,desc,'rgb')
  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'rgb')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'thumb')

  if not os.path.exists(path_):
    os.makedirs(path_)

  
  # path_ = os.path.split(file_)[0]

  # if path_ not in original_folder:
  #   original_folder.append(path_)

In [13]:
original_folder = []

for file_ in files:
  path_ = os.path.normpath(file_)
  paths_ = path_.split(os.sep)
  path_ = (os.sep).join(paths_[0:4])

  if path_ not in original_folder:
    original_folder.append(path_)

In [14]:
original_folder

['D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\2_밸리7H',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\3_마운틴너스리포장',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\4_마운틴1H',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\5_마운틴2H',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\6_마운틴3H',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\7_레이크2H',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\8_레이크3H',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H']

In [15]:
target_files = glob.glob(os.path.join(original_folder[3],'**/DJI_*.JPG'), recursive=True)
len(target_files)

618

In [70]:
InfoJsonArray = []

for idx, file_ in enumerate(target_files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  target_date = os.path.split(file_)[-1].split('_')[1][:8]
  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3]
  
  if len(info[5].split('_')) ==3 :
    desc = info[5].split('_')[-1]
  else:
    desc = "미기재"
  alt =  info[4].split('_')[-1]

  filesGrp = glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ os.path.split(file_)[1].split('_')[1][:-2]+"**_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)
  # print(filesGrp)
  file_info = getInfo(filesGrp, file_)

  file_info['info']['course'] = course
  file_info['info']['area'] = area
  file_info['info']['alt'] = alt
  file_info['info']['desc'] = desc
  file_info['info']['date'] = target_date


  # try:
  ndvi_img = getNDVIimg(filesGrp)
  rgb_img = getalignedRGB(filesGrp, ndvi_img)
  resized_ndvi = cv2.resize(ndvi_img, dsize=(2592,1944), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi','ndvi{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_ndvi)
  im.save(save_name)
  file_info['dest']['ndvi'] = save_name[9:]

  resized_rgb = cv2.resize(rgb_img , dsize=(648,486), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'rgb','rgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_rgb)
  im.save(save_name)
  file_info['dest']['rgb'] = save_name[9:]


  thumb_rgb = cv2.resize(rgb_img , dsize=(324,243), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'thumb','thumbrgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(thumb_rgb)
  im.save(save_name)
  file_info['dest']['thumb'] = save_name[9:]


  file_info['destFolder'] = ('/').join([target_date,area,desc])

  InfoJsonArray.append( file_info)



  
  print(save_name)

  print('{} out of {} files Completed'.format(idx, len(target_files)))

save_name = os.path.join(out_folder,course_id,target_date,area, 'data'+target_date+area+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)



  

C:\Users\yc463\AppData\Local\Temp\ipykernel_3424\891679340.py:18: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('RdYlGn')


D:\ToAWS\MGC002\20230823\3_마운틴너스리포장\잡초,방동사니,파대가리,사초과,충해,나방류,벤트,그린,토양병,레어리링\thumb\thumbrgb20230823114920_1286445033889_356650866389_165497000.JPG
0 out of 618 files Completed
D:\ToAWS\MGC002\20230823\3_마운틴너스리포장\잡초,방동사니,파대가리,사초과,충해,나방류,벤트,그린,토양병,레어리링\thumb\thumbrgb20230823114922_1286445032778_356650873611_165538000.JPG
1 out of 618 files Completed
D:\ToAWS\MGC002\20230823\3_마운틴너스리포장\잡초,방동사니,파대가리,사초과,충해,나방류,벤트,그린,토양병,레어리링\thumb\thumbrgb20230823114924_1286444993611_356650928889_165546000.JPG
2 out of 618 files Completed
D:\ToAWS\MGC002\20230823\3_마운틴너스리포장\잡초,방동사니,파대가리,사초과,충해,나방류,벤트,그린,토양병,레어리링\thumb\thumbrgb20230823114926_1286444965833_356650980278_165553000.JPG
3 out of 618 files Completed
D:\ToAWS\MGC002\20230823\3_마운틴너스리포장\잡초,방동사니,파대가리,사초과,충해,나방류,벤트,그린,토양병,레어리링\thumb\thumbrgb20230823114928_1286444956389_356650992500_165568000.JPG
4 out of 618 files Completed
D:\ToAWS\MGC002\20230823\3_마운틴너스리포장\잡초,방동사니,파대가리,사초과,충해,나방류,벤트,그린,토양병,레어리링\thumb\thumbrgb20230823114930_1286444941389_356651018611

Gather all data json files from Output Folder

In [21]:
dataJsonList = glob.glob(os.path.join(out_folder,course_id,target_date,'**/data*.json'), recursive=True)
dataJsonList

['D:\\ToAWS\\MGC002\\20230823\\10_스타트퍼팅그린\\data2023082310_스타트퍼팅그린.json',
 'D:\\ToAWS\\MGC002\\20230823\\3_마운틴너스리포장\\data202308233_마운틴너스리포장.json']

In [23]:
folderinDataJson = []
totalPhotoJson = []

for dataJson_ in dataJsonList:
  with open(dataJson_, "r") as _json:
    dataJson= json.load(_json)
  
  totalPhotoJson.extend(dataJson)

  for Json_ in dataJson:
    if Json_['destFolder'] not in folderinDataJson:
      folderinDataJson.append(Json_['destFolder'])




In [26]:
save_name = os.path.join(out_folder,course_id,target_date, 'photo.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(totalPhotoJson, final , ensure_ascii=False)

In [27]:
filestructure = []


for folder_ in folderinDataJson:
  fileinfo = {}
  fileinfo["path"] = folder_
  fileinfo["file_cnt"] = len([ x for x in totalPhotoJson if x["destFolder"] == folder_])
  fileinfo["labeled_file_cnt"] = 0
  filestructure.append(fileinfo)
  

In [28]:
save_name = os.path.join(out_folder,course_id,target_date, 'filepath_'+course_id + '_'+target_date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(filestructure, final , ensure_ascii=False)